### 総合演習データパイプライン

* 入力：CSVファイル ("C:\Users\Admin\ILP\Python-AI\raw_data.csv")
* 出力：

In [1]:
#必要なライブラリ
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('raw_data.csv',encoding='utf-8')
df.head()

,日付,予約件数,予約人数,来客数,総杯数,売上合計(円),ペールエール(本),ペールエール(円),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),黒ビール(本),黒ビール(円),フルーツビール(本),フルーツビール(円)
0,2024/04/01,NaN,NaN,21,33,"31,800",10,"10,000",10,"8,000",5,"4,500",5,"4,500",4,"4,800",NaN,NaN
1,2024/04/02,NaN,NaN,13,25,"23,300",8,"8,000",7,"5,600",3,"2,700",4,"3,600",2,"2,400",1,"1,000"
2,2024/04/03,NaN,NaN,20,36,"33,600",7,"7,000",8,"6,400",6,"5,400",6,"5,400",2,"2,400",7,"7,000"
3,2024/04/04,NaN,NaN,17,27,"24,600",6,"6,000",9,"7,200",4,"3,600",4,"3,600",1,"1,200",3,"3,000"
4,2024/04/05,2,5,29,61,"56,500",15,"15,000",18,"14,400",9,"8,100",8,"7,200",4,"4,800",7,"7,000"


In [12]:
# 欠損が「少ない行」を削除（5〜10行程度の欠損）
df = df.dropna(subset=[
    '来客数', '総杯数', 'ペールエール(本)', 'ラガー(本)'
])
# 欠損が「多すぎる列」を削除（90%以上欠損）
df = df.drop(columns=[
    '予約件数', '予約人数', '売上合計(円)', 'ペールエール(円)', '黒ビール(円)', 'フルーツビール(円)'
])
# 中程度の欠損（20%〜50%程度） → 平均や 0 で埋める
df['フルーツビール(本)'] = df['フルーツビール(本)'].fillna(df['フルーツビール(本)'].mean())
df['IPA(本)'] = df['IPA(本)'].fillna(df['IPA(本)'].mean())
df['ラガー(円)'] = df['ラガー(円)'].fillna(df['ラガー(円)'].mean())
df['IPA(円)'] = df['IPA(円)'].fillna(df['IPA(本)'].mean())
df['ホワイトビール(円)'] = df['ホワイトビール(円)'].fillna(df['ホワイトビール(円)'].mean())
df['ホワイトビール(本)'] = df['ホワイトビール(本)'].fillna(df['ホワイトビール(本)'].mean())
df['黒ビール(本)'] = df['黒ビール(本)'].fillna(df['黒ビール(本)'].mean())

In [ ]:
df.dtypes
# 日付を「YYYY/M/D」形式に変換
if '日付' in df.columns:
    df['年月日'] = pd.to_datetime(df['日付'], errors='coerce').dt.strftime('%Y/%-m/%-d')
    # Windows環境では「%-m」「%-d」が使えない場合があるので、下記も試せます
    # df['年月日'] = pd.to_datetime(df['日付'], errors='coerce').dt.strftime('%Y/%#m/%#d')
    # 必要なら元の「日付」列を削除
    # df = df.drop('日付', axis=1)
df_sales.head()

# DataFrameをJSONファイルとして保存（日付型も文字列で保存）
df.to_json('cleaned_data.json', orient='records', force_ascii=False, date_format='iso', indent=2)

日付            datetime64[ns]
来客数                  float64
総杯数                  float64
ペールエール(本)            float64
ラガー(本)               float64
ラガー(円)               float64
IPA(本)               float64
IPA(円)               float64
ホワイトビール(本)           float64
ホワイトビール(円)           float64
黒ビール(本)              float64
フルーツビール(本)           float64
dtype: object

In [ ]:
df_sales = pd.read_json('sales_data.json')
df_sales['日付'] = pd.to_datetime(df_sales['日付'], unit='ms')
df_sales

,日付,来客数,総杯数,ペールエール(本),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),黒ビール(本),フルーツビール(本)
0,2024-04-01,21,33,10,10,800,5.000000,3.545455,5.000000,900,4.000000,2.489247
1,2024-04-02,13,25,8,7,800,3.000000,3.545455,4.000000,900,2.000000,1.000000
2,2024-04-03,20,36,7,8,800,6.000000,3.545455,6.000000,900,2.000000,7.000000
3,2024-04-04,17,27,6,9,800,4.000000,3.545455,4.000000,900,1.000000,3.000000
4,2024-04-05,29,61,15,18,800,9.000000,3.545455,8.000000,900,4.000000,7.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
321,2025-03-26,14,22,6,7,800,3.000000,3.545455,2.000000,900,2.000000,2.000000
322,2025-03-27,2,4,1,1,800,1.000000,900.000000,3.167315,900,2.776824,1.000000
323,2025-03-28,33,74,25,20,800,12.000000,3.545455,9.000000,900,5.000000,3.000000
324,2025-03-29,1,2,1,1,800,3.545455,3.545455,3.167315,900,2.776824,2.489247


In [39]:
df_weather = pd.read_json('weather_data.json', encoding='utf-8', lines=True)
# 年月日列をdatetime型に変換
df_weather['年月日'] = pd.to_datetime(df_weather['年月日'], errors='coerce')
df_weather['年月日'] = df_weather['年月日'].dt.strftime('%Y/%-m/%-d')
# 00:00:00 を除き、年月日だけにする
# Windowsの場合は下記に変更（必要なら）
df_weather['年月日'] = pd.to_datetime(df_weather['年月日'], errors='coerce').dt.strftime('%Y/%#m/%#d')
df_weather = df_weather.rename(columns={"年月日": "日付"})
df_weather

,日付,平均気温(℃),日照時間(時間),降雪量合計(cm),平均湿度(％),最高気温(℃),最低気温(℃),10分間降水量の最大(mm),天気概況(昼：06時～18時),天気概況(夜：18時～翌日06時)
0,2024/4/1,14.6,3.8,0,73,19.7,11.8,1.5,曇時々雨一時晴,快晴
1,2024/4/2,14.2,10.6,0,50,20.6,8.4,0.0,晴後薄曇,曇
2,2024/4/3,13.9,0.0,0,83,16.6,12.3,1.5,雨時々曇,雨時々曇
3,2024/4/4,15.4,0.1,0,79,19.9,11.3,1.5,曇一時雨,曇後一時雨
4,2024/4/5,11.4,0.0,0,71,15.1,9.0,0.5,曇時々雨,曇時々雨
...,...,...,...,...,...,...,...,...,...,...
360,2025/3/27,17.9,5.6,0,69,22.4,12.2,0.0,薄曇一時晴,曇
361,2025/3/28,19.9,4.1,0,79,25.7,14.1,1.0,薄曇時々晴一時雨,雨時々曇一時晴
362,2025/3/29,7.6,0.0,0,90,14.3,4.3,1.0,雨,曇時々雨
363,2025/3/30,9.4,6.1,0,63,15.6,5.1,0.5,薄曇時々晴,曇時々雨


In [41]:
# df_salesとdf_weatherを「日付」列で結合
# まず「日付」列が両方に存在し、かつ同じフォーマットであることを確認
# df_weatherの「日付」列はすでに「YYYY/M/D」形式
# df_salesの「年月日」列を「日付」にリネーム
if '年月日' in df_sales.columns:
    df_sales = df_sales.rename(columns={'年月日': '日付'})
# 「日付」列をdatetime型に揃える
df_sales['日付'] = pd.to_datetime(df_sales['日付'], errors='coerce')
df_weather['日付'] = pd.to_datetime(df_weather['日付'], errors='coerce')
# 「日付」列で内部結合
merged_df = pd.merge(df_sales, df_weather, on='日付', how='inner')
merged_df.head()

,日付,来客数,総杯数,ペールエール(本),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),...,フルーツビール(本),平均気温(℃),日照時間(時間),降雪量合計(cm),平均湿度(％),最高気温(℃),最低気温(℃),10分間降水量の最大(mm),天気概況(昼：06時～18時),天気概況(夜：18時～翌日06時)
0,2024-04-01,21,33,10,10,800,5.0,3.545455,5.0,900,...,2.489247,14.6,3.8,0,73,19.7,11.8,1.5,曇時々雨一時晴,快晴
1,2024-04-02,13,25,8,7,800,3.0,3.545455,4.0,900,...,1.000000,14.2,10.6,0,50,20.6,8.4,0.0,晴後薄曇,曇
2,2024-04-03,20,36,7,8,800,6.0,3.545455,6.0,900,...,7.000000,13.9,0.0,0,83,16.6,12.3,1.5,雨時々曇,雨時々曇
3,2024-04-04,17,27,6,9,800,4.0,3.545455,4.0,900,...,3.000000,15.4,0.1,0,79,19.9,11.3,1.5,曇一時雨,曇後一時雨
4,2024-04-05,29,61,15,18,800,9.0,3.545455,8.0,900,...,7.000000,11.4,0.0,0,71,15.1,9.0,0.5,曇時々雨,曇時々雨


In [48]:
merged_df.to_feather('merged_data.feather')

In [46]:
import pyarrow.feather as feather

In [49]:
df = feather.read_feather('merged_data.feather')
df

,日付,来客数,総杯数,ペールエール(本),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),...,フルーツビール(本),平均気温(℃),日照時間(時間),降雪量合計(cm),平均湿度(％),最高気温(℃),最低気温(℃),10分間降水量の最大(mm),天気概況(昼：06時～18時),天気概況(夜：18時～翌日06時)
0,2024-04-01,21,33,10,10,800,5.000000,3.545455,5.000000,900,...,2.489247,14.6,3.8,0,73,19.7,11.8,1.5,曇時々雨一時晴,快晴
1,2024-04-02,13,25,8,7,800,3.000000,3.545455,4.000000,900,...,1.000000,14.2,10.6,0,50,20.6,8.4,0.0,晴後薄曇,曇
2,2024-04-03,20,36,7,8,800,6.000000,3.545455,6.000000,900,...,7.000000,13.9,0.0,0,83,16.6,12.3,1.5,雨時々曇,雨時々曇
3,2024-04-04,17,27,6,9,800,4.000000,3.545455,4.000000,900,...,3.000000,15.4,0.1,0,79,19.9,11.3,1.5,曇一時雨,曇後一時雨
4,2024-04-05,29,61,15,18,800,9.000000,3.545455,8.000000,900,...,7.000000,11.4,0.0,0,71,15.1,9.0,0.5,曇時々雨,曇時々雨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2025-03-26,14,22,6,7,800,3.000000,3.545455,2.000000,900,...,2.000000,20.4,10.2,0,45,25.9,14.9,0.0,晴,薄曇時々晴
302,2025-03-27,2,4,1,1,800,1.000000,900.000000,3.167315,900,...,1.000000,17.9,5.6,0,69,22.4,12.2,0.0,薄曇一時晴,曇
303,2025-03-28,33,74,25,20,800,12.000000,3.545455,9.000000,900,...,3.000000,19.9,4.1,0,79,25.7,14.1,1.0,薄曇時々晴一時雨,雨時々曇一時晴
304,2025-03-29,1,2,1,1,800,3.545455,3.545455,3.167315,900,...,2.489247,7.6,0.0,0,90,14.3,4.3,1.0,雨,曇時々雨


In [51]:
from sklearn.preprocessing import LabelEncoder

for col in ['天気概況(昼：06時～18時)', '天気概況(夜：18時～翌日06時)']:
    if col in df.columns:
        le = LabelEncoder()
        df[col + '_label'] = le.fit_transform(df[col].astype(str))
        # ラベル→元の天気概況の対応表を表示
        print(f'{col} のラベル対応表:')
        for label, name in zip(le.transform(le.classes_), le.classes_):
            print(f'{label}: {name}')
df.head()

天気概況(昼：06時～18時) のラベル対応表:
0: 大雨
1: 大雨、雷を伴う
2: 大雨一時曇
3: 大雨後時々曇
4: 快晴
5: 晴
6: 晴一時曇
7: 晴一時薄曇
8: 晴一時雨
9: 晴後一時曇
10: 晴後一時薄曇
11: 晴後一時薄曇、雷を伴う
12: 晴後一時雨
13: 晴後時々曇
14: 晴後時々曇一時雨
15: 晴後曇
16: 晴後曇一時雨
17: 晴後薄曇
18: 晴時々曇
19: 晴時々曇一時雨
20: 晴時々薄曇
21: 晴時々薄曇一時雨
22: 晴時々雨一時雪、あられを伴う
23: 晴時々雨一時雪、雷・ひょうを伴う
24: 曇
25: 曇一時晴
26: 曇一時雨
27: 曇後一時晴
28: 曇後一時雨
29: 曇後一時雨、みぞれを伴う
30: 曇後時々雨
31: 曇後晴
32: 曇後雨
33: 曇時々晴
34: 曇時々晴、雷を伴う
35: 曇時々晴後一時雨
36: 曇時々雨
37: 曇時々雨、雷を伴う
38: 曇時々雨一時晴
39: 曇時々雨後一時晴
40: 曇時々雨後一時晴、雷を伴う
41: 薄曇
42: 薄曇一時晴
43: 薄曇一時雨、雷を伴う
44: 薄曇後一時大雨、雷を伴う
45: 薄曇後一時晴
46: 薄曇後一時雨
47: 薄曇後晴
48: 薄曇時々晴
49: 薄曇時々晴一時雨
50: 薄曇時々雨
51: 雨
52: 雨一時曇
53: 雨一時曇後晴
54: 雨後みぞれ一時雪
55: 雨後一時曇
56: 雨後一時霧雨
57: 雨後時々曇
58: 雨後曇
59: 雨後薄曇
60: 雨時々曇
61: 雨時々曇後晴
天気概況(夜：18時～翌日06時) のラベル対応表:
0: 大雨
1: 大雨時々曇、雷を伴う
2: 快晴
3: 晴
4: 晴一時曇
5: 晴一時薄曇
6: 晴後一時曇
7: 晴後一時薄曇
8: 晴後曇
9: 晴後曇一時雨
10: 晴後曇時々雨、あられを伴う
11: 晴後薄曇
12: 晴時々曇
13: 晴時々曇一時雨
14: 晴時々薄曇
15: 晴時々薄曇一時雨、雷を伴う
16: 曇
17: 曇、雷を伴う
18: 曇一時みぞれ
19: 曇一時大雨後時々晴、雷を伴う
20: 曇一時晴
21: 曇一時雨
22: 曇一時雨、雷を伴う
23: 曇一時雨後一時晴、雷を伴う
24: 曇一時雨後晴、雷を伴う
25: 曇一時霧雨
26:

,日付,来客数,総杯数,ペールエール(本),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),...,日照時間(時間),降雪量合計(cm),平均湿度(％),最高気温(℃),最低気温(℃),10分間降水量の最大(mm),天気概況(昼：06時～18時),天気概況(夜：18時～翌日06時),天気概況(昼：06時～18時)_label,天気概況(夜：18時～翌日06時)_label
0,2024-04-01,21,33,10,10,800,5.0,3.545455,5.0,900,...,3.8,0,73,19.7,11.8,1.5,曇時々雨一時晴,快晴,38,2
1,2024-04-02,13,25,8,7,800,3.0,3.545455,4.0,900,...,10.6,0,50,20.6,8.4,0.0,晴後薄曇,曇,17,16
2,2024-04-03,20,36,7,8,800,6.0,3.545455,6.0,900,...,0.0,0,83,16.6,12.3,1.5,雨時々曇,雨時々曇,60,53
3,2024-04-04,17,27,6,9,800,4.0,3.545455,4.0,900,...,0.1,0,79,19.9,11.3,1.5,曇一時雨,曇後一時雨,26,27
4,2024-04-05,29,61,15,18,800,9.0,3.545455,8.0,900,...,0.0,0,71,15.1,9.0,0.5,曇時々雨,曇時々雨,36,34
